## Created by yunsuxiaozi 2024/3/24

### 相信对于大多数接触机器学习算法的人来说,线性回归算法应该是第一个接触的。

### 线性回归是最简单的算法,但是并不代表它很差。在Kaggle最近举办的<a href="https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability">Home Credit - Credit Risk Model Stability</a> 比赛中,我使用线性回归打败了大约40%的选手。<a href="https://www.kaggle.com/code/yunsuxiaozi/home-credit-linearregression-is-all-you-need">Home Credit LinearRegression is all you need.</a> 所以在这里,我将介绍一下线性回归算法的数学原理以及代码实现,并用一个数据集来比较我的算法和sklearn库的算法.

### 线性回归就是$\hat{y}=XW+b$.其中$\hat{y}$是预测值,y是真实值.

### 我们先来说明一下公式中出现的字母:

- X:这是数据的特征.shape:(len(dataset),len(features))

- y:这是需要预测的target.shape:(len(dataset),y.shape[1]),其中y.shape[1]一般为1.

- W:是线性回归算法中X里每个特征的权重.shape:(len(features),y.shape[1]).

- b:是线性回归的偏置.shape:(1,y.shape[1]).

### 关于线性回归,如果数据量太小,或者说矩阵很小,可以用公式直接求解,想看公式推导可以看我以前写的文章<a href="https://zhuanlan.zhihu.com/p/581284145">n维线性回归模型</a>。如果数据量太大,公式求解就显得不现实了,一方面公式中有矩阵求逆这个(On^3)的步骤,时间上不划算,另一方面如何存储这么大的矩阵也是个问题.这时就需要用迭代的方式来求解了.

### 回归任务中常用的损失函数是$MSE=\frac{(\hat{y}-y)^2}{n}=\frac{(XW+b-y)^2}{n}$,我们的目的就是让MSE最小.这里使用的迭代算法就是梯度下降法.此处梯度是高数的内容,在一元函数里就是导数.

### 这里举个例子:$MSE=x^2$,我们随便设置一个初始点$x=1$,我们想到达最低点$x=0$,此处的导数为$2x=2$.如果往正方向走两步明显是往高处走,所以我们应该往导数的反方向走。如果我们往负数方向走长度2,会到达-1,此时已经经过最低点了,所以我们还需要控制步长,比如在导数前面乘一个$\frac{1}{2}$,让它往负数方向走长度1,这样才能到达最低点0.这个控制步长的$\frac{1}{2}$被叫做学习率.从初始位置每次沿着导数反方向走一定的距离最终到达最低点附近的方法就是梯度下降法.


### 在线性回归的损失函数$MSE=\frac{(XW+b-y)^2}{n}$中,X和y是训练数据和标签,这些是固定的,n是数据的数量,也是确定的.真正的参数是$W,b$,我们需要做的是对它们求偏导.

### $MSE'_W=\frac{1}{n}(2WXX+2Xb-2yX)=\frac{1}{n}(2X(WX+b-y))=\frac{1}{n}(2X(\hat{y}-y))$

### $MSE'_b=\frac{1}{n}(2XW+2b-2y)=\frac{2}{n}(\hat{y}-y)$

### 梯度下降法的迭代公式就是$W=W-lr*(\frac{1}{n})(2X(\hat{y}-y),b=b-lr*\frac{2}{n}(\hat{y}-y)$

### 注:笔者不是数学专业的学生,对公式细节方面可能不会特别重视,矩阵的转置包括矩阵的先后顺序在公式里没有管。

### 至于算法实现方面,由于我们考虑到矩阵可能会很大,需要将数据集分批次进行训练,比如我有一百万数据,我一次用1万数据进行迭代,迭代100次完成一个epoch.

### 下面我们用一个数据集来实战一下吧:<a href="https://www.kaggle.com/datasets/parasharmanas/world-press-freedom-index-2023-trends">World Press Freedom Index 2023 Trends</a>.这个数据集是我一年前学习线性回归的时候使用的,那时还是2022年版.下面将使用这个数据集来对我的算法和python开源的算法进行比较。

#### 导入一些python库并固定随机种子

In [1]:
import pandas as pd#处理表格数据的库
import numpy as np#进行矩阵运算的库
import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

#### 导入数据集.

In [2]:
dataset=pd.read_csv("./RSB_DataSet.csv",encoding='gbk')
print(f"len(dataset):{len(dataset)}")
dataset.head()

len(dataset):180


,Ranking,Country,Score,Political Context,Economic Context,Legal Framework,Safety Score,Sociocultural Context,Difference In Position From 2022
0,1,Norway,95.18,96.54,92.46,94.92,95.98,95.98,0
1,2,Ireland,89.91,93.91,82.11,82.55,96.94,94.03,4
2,3,Denmark,89.48,91.95,85.17,87.50,95.00,87.78,-1
3,4,Sweden,88.15,92.58,86.08,88.74,84.72,88.64,-1
4,5,Finland,87.94,91.55,83.80,84.86,90.35,89.17,0


#### 我们看到中国在180个国家中位列倒数第2,仅次于朝鲜,可以说是遥遥领先.

In [3]:
dataset.tail()

,Ranking,Country,Score,Political Context,Economic Context,Legal Framework,Safety Score,Sociocultural Context,Difference In Position From 2022
175,176,Turkmenistan,25.82,23.25,20.39,29.62,46.53,9.32,1
176,177,Iran,24.81,33.67,30.51,20.17,23.62,16.06,1
177,178,Vietnam,24.58,23.75,17.16,18.40,30.66,32.95,-4
178,179,China,22.97,26.06,29.51,17.36,24.87,17.07,-4
179,180,North Korea,21.72,26.56,21.57,22.64,33.25,4.60,0


#### 这里是我手动实现的线性回归算法。

In [4]:
class LinearRegression():
    def __init__(self,n_iterators=500,batch_size=100000,lr=0.02,metric='MSE'):
        self.n_iterators=n_iterators#迭代多少次
        self.batch_size=batch_size#用迭代算法每次传入的批次数量
        self.lr=lr#学习率
        self.metric=metric#用什么评估指标来评估模型的效果
        self.mean=0.0#均值
        self.std=1.0#方差
        
    def fit(self,train_X,train_y,is_standardization=True):
        """
        train_X.shape:(num_data,num_features)
        train_y.shape:(num_data,y.shape[1])
        self.W.shape:(num_features,y.shape[1])
        self.b.shape:(1,y.shape[1])
        """
        if len(train_y.shape)==1:
            train_y=train_y.reshape(-1,1)
        self.W=np.random.randn(train_X.shape[1],train_y.shape[1])
        self.b=np.random.randn(1,train_y.shape[1])
        if is_standardization:#如果对数据进行标准化的话,求出均值和方差
            self.mean=train_X.mean(axis=0)
            self.std=train_X.std(axis=0)
        train_X=(train_X-self.mean)/self.std
        for iterator in range(self.n_iterators):  
            random_number=np.arange(len(train_X))
            np.random.shuffle(random_number)
            train_X=train_X[random_number]
            train_y=train_y[random_number]
            losses=[]
            for idx in range(0,len(train_X),self.batch_size):
                train_X1=train_X[idx:idx+self.batch_size]
                train_y1=train_y[idx:idx+self.batch_size]
                train_pred=train_X1.dot(self.W)+self.b
                loss=self.loss(train_y1,train_pred)
                temp_b=np.tile(self.b,(train_X1.shape[0],1))
                #更新参数
                self.W=self.W-2*self.lr*(train_X1.T).dot(train_pred-train_y1)/len(train_X1)
                self.b=self.b-2*self.lr*np.mean(train_pred-train_y1,axis=0)
                losses.append(loss)
            #print(f"iterator:{iterator},loss:{np.mean(np.array(losses))}")
            
    def predict(self,test_X):
        test_X=(test_X-self.mean)/self.std
        test_pred=np.zeros((len(test_X),self.b.shape[1]))
        for idx in range(0,len(test_X),self.batch_size):
            test_X1=test_X[idx:idx+self.batch_size]
            test_pred[idx:idx+len(test_X1)]=test_X1.dot(self.W)+self.b
        return test_pred.reshape(-1)
    
    def loss(self,y_true,y_pred):
        if self.metric.lower()=='mse':
            return self.MSE(y_true,y_pred)
        elif self.metric.lower()=='rmse':
            return self.RMSE(y_true,y_pred)
        elif self.metric.lower()=='mae':
            return self.MAE(y_true,y_pred)
    def MSE(self,y_true,y_pred):
        return np.mean((y_true-y_pred)**2)
    def MAE(self,y_true,y_pred):
        return np.mean(np.abs(y_true-y_pred))
    def RMSE(self,y_true,y_pred):
        return np.sqrt(np.mean((y_true-y_pred)**2))

#### 我们按照8:2的比例划分训练集和测试集,并用线性回归模型进行训练和预测,MSE约为0.05.

In [5]:
X=dataset[['Political Context', 'Economic Context','Legal Framework', 
           'Safety Score', 'Sociocultural Context']].values
y=dataset['Score'].values
#划分训练集和测试集的函数
def train_test_split(dataX,datay,shuffle=True,percentage=0.8):
    """
    将训练数据X和标签y以numpy.array数组的形式传入
    划分的比例定为训练集:测试集=8:2
    """
    if shuffle:
        random_num=[index for index in range(len(dataX))]
        np.random.shuffle(random_num)
        dataX=dataX[random_num]
        datay=datay[random_num]
    split_num=int(len(dataX)*percentage)
    train_X=dataX[:split_num]
    train_y=datay[:split_num]
    test_X=dataX[split_num:]
    test_y=datay[split_num:]
    return train_X,train_y,test_X,test_y
def MSE(y_true,y_pred):
    return np.mean((y_true-y_pred)**2)
train_X,train_y,test_X,test_y=train_test_split(X,y,shuffle=True,percentage=0.8)
model=LinearRegression()
model.fit(train_X,train_y)
test_pred=model.predict(test_X)
print(f"MSE:{MSE(test_y,test_pred)},model.W:{model.W},model.b:{model.b}")

MSE:0.056606730829491775,model.W:[[3.4523525 ]
 [3.16890065]
 [3.91554814]
 [4.60958612]
 [4.10433146]],model.b:[[58.61687492]]


#### 接下来调用python的sklearn里的线性回归模型,我们可以看到MSE大约为0.05.我们可以看到两种算法效果还是差不多的.至于最后找到的参数不同是因为算法在实现细节上是不同的.

In [6]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(train_X,train_y)
test_pred=model.predict(test_X)
print(f"MSE:{MSE(test_y,test_pred)},model.coef_:{model.coef_}, model.intercept_:{model.intercept_} ")

MSE:0.05745845678339052,model.coef_:[0.20004491 0.19992727 0.19996543 0.20000271 0.20004921], model.intercept_:-0.0003447826392246611 


#### 如果你觉得我的notebook对你有用的话,请给我点个赞,谢谢.如果你对我的notebook有什么想法或者建议也欢迎提出来,以便于我更好的完善.